In [ ]:
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd
!pip install asyncio
!pip install nest_asyncio
import asyncio
import nest_asyncio

nest_asyncio.apply()  # Apply the patch to allow running asyncio in Jupyter

async def wait_60_seconds():
    print("Waiting for 60 seconds...")
    await asyncio.sleep(60)
    print("60 seconds have passed. Now executing the code.")

async def retrieve_data(stock):
    apikey = "zDF6DwutU82qPPDf1M7GH2HsoafyDy8q29M7fGNn"
    
    # First API Call
    url = "https://yfapi.net/v6/finance/quote"
    querystr = {"symbols": stock, "region": "US", "lang": "en"}
    headers = {'x-api-key': apikey}
    response = requests.request("GET", url, headers=headers, params=querystr)
    stock_json = response.json()

    if ("quoteResponse" in stock_json
            and "result" in stock_json["quoteResponse"]
            and stock_json["quoteResponse"]["result"][0].get("triggerable") == False
    ):
        print("Error: Stock ticker or information does not exist")
    else:
        print(stock_json)

    # Second API Call
    url2 = "https://yfapi.net/v11/finance/quoteSummary/" + stock
    querystr2 = {"region": "US", "lang": "en", "modules": "financialData"}
    response2 = requests.request("GET", url2, headers=headers, params=querystr2)
    stock_json2 = response2.json()

    if ("quoteResponse" in stock_json2
            and "result" in stock_json2["quoteResponse"]
            and stock_json2["quoteResponse"]["result"][0].get("triggerable") == False
    ):
        print("Error: Stock ticker or information does not exist")
    else:
        print(stock_json2)

    name = str(stock_json["quoteResponse"]['result'][0]["longName"])
    price = str(stock_json['quoteResponse']['result'][0]['regularMarketPrice'])
    grossprofits = str(stock_json2["quoteSummary"]['result'][0]['financialData']['grossProfits']['fmt'])
    targetmeanprice = str(stock_json2["quoteSummary"]['result'][0]['financialData']['targetMeanPrice']['fmt'])
    totalcash = str(stock_json2["quoteSummary"]['result'][0]['financialData']['totalCash']['fmt'])

    print("Stock: " + str(stock) + " Company: " + str(name) + " Price: " + str(price) +
          ' Gross Profits: ' + str(grossprofits) + ' Target Mean Price ' + str(targetmeanprice) +
          ' Total Cash: ' + totalcash)

async def main():
    columns = ["stock", "company", "grossprofits", "targetmeanprice", "totalCash"]
    df = pd.DataFrame(columns=columns)
    stock = input("Enter stock ticker: ")
    print("The stock that will be searched is " + stock)

    for _ in range(60):  # Loop 60 times for 60 minutes
        await retrieve_data(stock)
        await wait_60_seconds()

# Run the main execution loop
if __name__ == "__main__":
    asyncio.run(main())


Enter stock ticker:  Aapl


The stock that will be searched is Aapl
{'quoteResponse': {'result': [{'language': 'en-US', 'region': 'US', 'quoteType': 'EQUITY', 'triggerable': True, 'quoteSourceName': 'Nasdaq Real Time Price', 'market': 'us_market', 'regularMarketChangePercent': -0.33864066, 'regularMarketDayRange': '192.205 - 194.76', 'regularMarketPreviousClose': 193.42, 'bid': 192.81, 'ask': 192.83, 'bidSize': 8, 'askSize': 8, 'messageBoardId': 'finmb_24937', 'currency': 'USD', 'trailingAnnualDividendYield': 0.0048598903, 'epsTrailingTwelveMonths': 6.13, 'epsForward': 7.15, 'epsCurrentYear': 6.56, 'priceEpsCurrentYear': 29.384909, 'sharesOutstanding': 15552799744, 'bookValue': 3.997, 'fiftyDayAverage': 180.0912, 'fiftyDayAverageChange': 12.673798, 'fiftyDayAverageChangePercent': 0.07037433, 'twoHundredDayAverage': 175.6501, 'twoHundredDayAverageChange': 17.114899, 'twoHundredDayAverageChangePercent': 0.097437456, 'marketCap': 2998035546112, 'forwardPE': 26.96014, 'priceToBook': 48.22742, 'sourceInterval': 15, 'e